In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/burgers-clean")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/burgers-clean/versions/5


# pinn class

In [2]:
import torch
import torch.nn as nn
import numpy as np


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.lambda2 = nn.Parameter(
            torch.tensor([0.01], dtype=torch.float32, device="cuda")
        )
        self.lambda1 = nn.Parameter(
            torch.tensor([1.0], dtype=torch.float32, device="cuda")
        )
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.optimizer.param_groups[0]["params"].append(self.lambda1)
        self.optimizer.param_groups[0]["params"].append(self.lambda2)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred,
            g,
            torch.ones([xtrain.shape[0], 1]).to("cuda"),
            retain_graph=True,
            create_graph=True,
        )[0]
        u_xx_tt = torch.autograd.grad(
            u_x_t, g, torch.ones(xtrain.shape).to("cuda"), create_graph=True
        )[0]
        u_x = u_x_t[:, [0]]
        u_t = u_x_t[:, [1]]
        u_xx = u_xx_tt[:, [0]]
        return self.loss(u_t + (self.lambda1 * u_pred * u_x) - (self.lambda2 * u_xx), fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, Lambda2 (Nu) {self.lambda2.item()}")
        return loss.item()


model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): Tanh()
    (18): Linear(in_features=20, out_features=20, bias=True)
    (19): Tanh()
    (20): Linear(in_features=20, out_features=1, bias=True)
  )
  (loss): MSELoss()
)


# load data

In [12]:
import torch
import numpy as np
def loadAndPrep(u):
    x = [i for i in range(201)]
    t = [i for i in range(1024)]
    x = torch.tensor(x, dtype=torch.float32)
    t = torch.tensor(t, dtype=torch.float32)
    u = torch.tensor(u, dtype=torch.float32)
    X, T = np.meshgrid(t, x)
    xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    lb = xtrue[0]
    ub = xtrue[-1]
    idx = np.random.choice(201*1024, 10000, replace=False)
    xtrain = xtrue[idx, :]
    utrain = u.flatten()[idx][:, None]
    device = torch.device("cuda")
    xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
    xtrue = torch.tensor(xtrue, dtype=torch.float32, device=device)
    utrain = utrain.to(device)
    utrue = u.flatten()[:, None]
    return xtrain, xtrue, utrain, utrue

In [13]:
import gc
def trainAndLog(u):
    xtrain, _, utrain, _ = loadAndPrep(u)
    model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
    loss = model.train_model(xtrain, utrain, epochs=10000)
    l = model.lambda2.item()
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return l, loss

In [9]:
import h5py
import random
random.seed(69)
d = {}
with h5py.File("/root/.cache/kagglehub/datasets/shusrith/burgers-clean/versions/5/simulation_data.h5", "r") as f:
    a = random.choices(list(f.keys()), k=100)
    n = 0
    for i in a[:2]:
        print(n)
        d[i] = []
        d[i].append({"epsilon" : f[i]["epsilon"][()]})
        uclean = f[i]["clean"][:]
        pred, loss = trainAndLog(uclean)
        d[i].append({"clean" : [{"predicted" : pred, "loss" : loss}]})
        unoisy = f[i]["noisy"][:]
        pred, loss = trainAndLog(unoisy)
        d[i].append({"noisy": [{"predicted" : pred, "loss" : loss}]})
        n += 1

0


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0, Loss 0.5056760907173157, Lambda2 (Nu) 0.010001664981245995
Epoch 1000, Loss 0.48108845949172974, Lambda2 (Nu) 0.1740458458662033
Epoch 2000, Loss 0.47685861587524414, Lambda2 (Nu) 0.2158416509628296
Epoch 3000, Loss 0.4917241036891937, Lambda2 (Nu) -0.11704258620738983
Epoch 4000, Loss 0.49235570430755615, Lambda2 (Nu) -0.04915159195661545
Epoch 5000, Loss 0.4916537404060364, Lambda2 (Nu) -0.015493352897465229
Epoch 6000, Loss 0.4787796139717102, Lambda2 (Nu) -0.028635742142796516
Epoch 7000, Loss 0.49161678552627563, Lambda2 (Nu) 0.0005129274795763195
Epoch 8000, Loss 0.49152615666389465, Lambda2 (Nu) 0.009952335618436337
Epoch 9000, Loss 0.4948279559612274, Lambda2 (Nu) 0.02076083794236183
Epoch 0, Loss 0.5593153238296509, Lambda2 (Nu) 0.009998958557844162
Epoch 1000, Loss 0.5098941326141357, Lambda2 (Nu) 0.018785996362566948
Epoch 2000, Loss 0.5100711584091187, Lambda2 (Nu) 0.018156906589865685
Epoch 3000, Loss 0.5064123272895813, Lambda2 (Nu) 0.00874553993344307
Epoch 4000

In [6]:
!ls /kaggle/input

In [10]:
import json
with open("results.json", "w") as f:
    json.dump(d, f)